In [2]:
import pandas as pd
import numpy as np

In [9]:
m_detailed_results = pd.read_csv('data/men/MRegularSeasonDetailedResults.csv')
m_detailed_results['Side'] = 'Men'
w_detailed_results = pd.read_csv('data/women/WRegularSeasonDetailedResults.csv')
w_detailed_results['Side'] = 'Women'

detailed_results = pd.concat([m_detailed_results, w_detailed_results], ignore_index=True)

In [10]:
detailed_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Side
0,2003,10,1104,68,1328,62,N,0,27,58,...,16,22,10,22,8,18,9,2,20,Men
1,2003,10,1272,70,1393,63,N,0,26,62,...,9,20,20,25,7,12,8,6,16,Men
2,2003,11,1266,73,1437,61,N,0,24,58,...,14,23,31,22,9,12,2,5,23,Men
3,2003,11,1296,56,1457,50,N,0,18,38,...,8,15,17,20,9,19,4,3,23,Men
4,2003,11,1400,77,1208,71,N,0,30,61,...,17,27,21,15,12,10,7,1,14,Men
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198369,2025,106,3242,63,3416,58,H,0,17,46,...,5,11,15,21,11,12,4,2,21,Women
198370,2025,106,3329,68,3428,64,A,0,23,63,...,9,16,8,18,16,11,8,6,20,Women
198371,2025,106,3349,72,3194,39,H,0,30,63,...,16,23,4,25,5,10,4,3,9,Women
198372,2025,106,3378,70,3150,52,A,0,25,59,...,11,14,7,24,5,15,8,2,15,Women


In [11]:
detailed_results['LLoc'] = detailed_results['WLoc'].apply(lambda x: 'A' if x == 'H' else ('H' if x == 'A' else 'N'))

winner_rows = detailed_results.copy()
loser_rows = detailed_results.copy()

for col in detailed_results.columns.tolist():
    if col[0] == 'W':
        stat = col[1:]
        winner_rows[stat] = winner_rows['W' + stat]
        winner_rows['Opp' + stat] = winner_rows['L' + stat]
        winner_rows['Win'] = 1
        loser_rows[stat] = loser_rows['L' + stat]
        loser_rows['Opp' + stat] = loser_rows['W' + stat]
        loser_rows['Win'] = 0

        winner_rows = winner_rows.drop(columns=['W' + stat, 'L' + stat])
        loser_rows = loser_rows.drop(columns=['W' + stat, 'L' + stat])

games_expanded_detailed = pd.concat([winner_rows, loser_rows], ignore_index=True)
games_expanded_detailed.columns

Index(['Season', 'DayNum', 'NumOT', 'Side', 'TeamID', 'OppTeamID', 'Win',
       'Score', 'OppScore', 'Loc', 'OppLoc', 'FGM', 'OppFGM', 'FGA', 'OppFGA',
       'FGM3', 'OppFGM3', 'FGA3', 'OppFGA3', 'FTM', 'OppFTM', 'FTA', 'OppFTA',
       'OR', 'OppOR', 'DR', 'OppDR', 'Ast', 'OppAst', 'TO', 'OppTO', 'Stl',
       'OppStl', 'Blk', 'OppBlk', 'PF', 'OppPF'],
      dtype='object')

In [12]:
games_expanded_detailed['Possessions'] = (games_expanded_detailed['FGA'] - games_expanded_detailed['OR'] + games_expanded_detailed['TO'] + 0.44 * games_expanded_detailed['FTA'])
games_expanded_detailed['OppPossessions'] = (games_expanded_detailed['OppFGA'] - games_expanded_detailed['OppOR'] + games_expanded_detailed['OppTO'] + 0.44 * games_expanded_detailed['OppFTA'])

for team in ["", 'Opp']:
    if team == 'Opp':
        other = ""
    else:
        other = 'Opp'
    games_expanded_detailed[team + 'FGPercent'] = np.where(games_expanded_detailed[team + 'FGA'] == 0, None, games_expanded_detailed[team + 'FGM'] / games_expanded_detailed[team + 'FGA'])
    games_expanded_detailed[team + 'FG3Percent'] = np.where(games_expanded_detailed[team + 'FGA3'] == 0, None, games_expanded_detailed[team + 'FGM3'] / games_expanded_detailed[team + 'FGA3'])
    games_expanded_detailed[team + 'FTPercent'] = np.where(games_expanded_detailed[team + 'FTA'] == 0, None, games_expanded_detailed[team + 'FTM'] / games_expanded_detailed[team + 'FTA'])
    games_expanded_detailed[team + 'ORPercent'] = np.where((games_expanded_detailed[team + 'OR'] == 0) & (games_expanded_detailed[other + 'DR'] == 0), None, games_expanded_detailed[team + 'OR'] / (games_expanded_detailed[team + 'OR'] + games_expanded_detailed[other + 'DR']))
    games_expanded_detailed[team + 'AstPercent'] = np.where(games_expanded_detailed[team + 'FGM'] == 0, None, games_expanded_detailed[team + 'Ast'] / games_expanded_detailed[team + 'FGM'])

    games_expanded_detailed[team + 'PointsPerPoss'] = np.where(games_expanded_detailed[team + 'Possessions'] == 0, None, games_expanded_detailed[team + 'Score'] / games_expanded_detailed[team + 'Possessions'])
    games_expanded_detailed[team + 'TOPerPoss'] = np.where(games_expanded_detailed[team + 'Possessions'] == 0, None, games_expanded_detailed[team + 'TO'] / games_expanded_detailed[team + 'Possessions'])
    games_expanded_detailed[team + 'StlPerPoss'] = np.where(games_expanded_detailed[other + 'Possessions'] == 0, None, games_expanded_detailed[team + 'Stl'] / games_expanded_detailed[other + 'Possessions'])
    games_expanded_detailed[team + 'BlkPerPoss'] = np.where(games_expanded_detailed[other + 'Possessions'] == 0, None, games_expanded_detailed[team + 'Blk'] / games_expanded_detailed[other + 'Possessions'])
    games_expanded_detailed[team + 'PFPerPoss'] = np.where(games_expanded_detailed[team + 'Possessions'] == 0, None, games_expanded_detailed[team + 'Stl'] / (games_expanded_detailed[team + 'Possessions'] + games_expanded_detailed[other + 'Possessions']))





games_expanded_detailed

,Season,DayNum,NumOT,Side,TeamID,OppTeamID,Win,Score,OppScore,Loc,...,OppFGPercent,OppFG3Percent,OppFTPercent,OppORPercent,OppAstPercent,OppPointsPerPoss,OppTOPerPoss,OppStlPerPoss,OppBlkPerPoss,OppPFPerPoss
0,2003,10,0,Men,1104,1328,1,68,62,N,...,0.415094,0.2,0.727273,0.294118,0.363636,0.877193,0.254669,0.120128,0.026695,0.061813
1,2003,10,0,Men,1272,1393,1,70,63,N,...,0.358209,0.25,0.45,0.416667,0.291667,0.929204,0.176991,0.117028,0.087771,0.058754
2,2003,11,0,Men,1266,1437,1,73,61,N,...,0.30137,0.115385,0.608696,0.54386,0.409091,0.951341,0.187149,0.031368,0.078419,0.01564
3,2003,11,0,Men,1296,1457,1,56,50,N,...,0.367347,0.272727,0.533333,0.472222,0.5,0.868056,0.329861,0.069396,0.052047,0.03471
4,2003,11,0,Men,1400,1208,1,77,71,N,...,0.387097,0.375,0.62963,0.488372,0.5,1.129135,0.159033,0.109856,0.015694,0.055292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396743,2025,106,0,Women,3416,3242,0,58,63,A,...,0.369565,0.235294,0.892857,0.192308,0.529412,0.964483,0.183711,0.06169,0.077113,0.030731
396744,2025,106,0,Women,3428,3329,0,64,68,H,...,0.365079,0.26087,0.888889,0.5,0.391304,0.97254,0.243135,0.088183,0.044092,0.043491
396745,2025,106,0,Women,3194,3349,0,39,72,A,...,0.47619,0.416667,0.5,0.166667,0.7,1.03211,0.143349,0.084364,0.070304,0.042589
396746,2025,106,0,Women,3150,3378,0,52,70,H,...,0.423729,0.235294,0.888889,0.25,0.6,0.987028,0.169205,0.124723,0.041574,0.062902


In [15]:
# Function to calculate the offensive and defensive rating adjustments
def adjust_ratings(games_expanded, creation_ratings, prevention_ratings, metric, metric_average, iterations=1, k=.01):
    for _ in range(iterations):
        #print(defensive_ratings[1101])
        #TODO weight late season games more heavily here maybe? Like include those rows more times?
        #Manual injury updates?
        #TODO can also probably produce error metrics here to generate distributions for each of these stats where we could do monte carlo to simulate a game enough to know the winner
        #TODO some way of tracking if we've iterated enough for things to stabilize, some kind of max adj, probably just save the whole frame as prev or something
        for _, game in games_expanded.iterrows():
            team = game['TeamID']
            opponent = game['OppTeamID']

            # Calculate expected value of metric for both teams
            #team's expected points is a combination of how good they are at creating points and how bad the opponent is at preventing points
            expected_team_creation = creation_ratings[team] * (prevention_ratings[opponent] / metric_average)
            expected_opponent_creation = creation_ratings[opponent] * (prevention_ratings[team] / metric_average)

            actual_team_metric = game[metric]
            actual_opponent_metric = game['Opp' + metric]

            if actual_team_metric is not None:
                team_rating_adjustment = k * (actual_team_metric - expected_team_creation)
            if actual_opponent_metric is not None:
                opponent_rating_adjustment = k * (actual_opponent_metric - expected_opponent_creation)

            creation_ratings[team] += team_rating_adjustment
            prevention_ratings[team] += opponent_rating_adjustment
            creation_ratings[opponent] += opponent_rating_adjustment
            prevention_ratings[opponent] += team_rating_adjustment

    return creation_ratings, prevention_ratings

In [23]:
metrics = ['Score', 'Possessions', 'FGPercent', 'FG3Percent', 'FTPercent', 'ORPercent', 'AstPercent', 'PointsPerPoss', 'TOPerPoss', 'StlPerPoss', 'BlkPerPoss', 'PFPerPoss']

#TODO Really we should hold out the games one at a time to get ratings for the whole college basketball landscape without that game, then predict the score of the game based on those...
all_seasons_men_team_ratings = []
all_seasons_women_team_ratings = []


In [24]:
#Decided to do these differently rather than just call above function on a list of mens and womens teams primarily because it just seems wrong to have two subgroups passed in. But more practically the metric averages would be noticeably different so normalization is definitely better if we do them separately
for side in ['Men', 'Women']:
    side_games_expanded = games_expanded_detailed[games_expanded_detailed['Side'] == side].copy()
    #seasons = [2025]
    seasons = side_games_expanded['Season'].unique()
    for y in seasons:
        all_metrics = None
        for m in metrics:
            print('Side', side, 'Running Season', y, 'Metric', m)
            year_games_expanded = side_games_expanded[side_games_expanded['Season'] == y].copy()
            year_teams = year_games_expanded['TeamID'].unique()

            metric_average = year_games_expanded[m].mean()
            print(metric_average)
            creation_ratings = {team: metric_average for team in year_teams}  # Starting offensive rating of 100 for all teams
            prevention_ratings = {team: metric_average for team in year_teams}  # Starting defensive rating of 100 for all teams


            creation_ratings, prevention_ratings = adjust_ratings(year_games_expanded, creation_ratings, prevention_ratings, m, metric_average, iterations = 10, k=.01)


            # Convert to DataFrame
            year_metric_team_ratings = pd.DataFrame({
                'TeamID': list(creation_ratings.keys()),
                m + 'CreationRating': list(creation_ratings.values()),
                m + 'PreventionRating': list(prevention_ratings.values())
            })

            if all_metrics is None:
                all_metrics = year_metric_team_ratings
            else:
                all_metrics = all_metrics.merge(year_metric_team_ratings, on = 'TeamID')


        all_metrics['Season'] = y

        #print(year_team_ratings.head())
        if side == 'Men':
            all_seasons_men_team_ratings.append(all_metrics)
        else:
            all_seasons_women_team_ratings.append(all_metrics)


#Higher prevention rating means the other team gets more of this metric against you
all_team_men_stat_ratings_df = pd.concat(all_seasons_men_team_ratings, ignore_index=True)
all_team_men_stat_ratings_df.to_csv("MStatRatings.csv", index=False)

all_team_women_stat_ratings_df = pd.concat(all_seasons_women_team_ratings, ignore_index=True)
all_team_women_stat_ratings_df.to_csv("WStatRatings.csv", index=False)

all_team_women_stat_ratings_df

Side Men Running Season 2003 Metric Score
69.77599653379549
Side Men Running Season 2003 Metric Possessions
68.36460571923745
Side Men Running Season 2003 Metric FGPercent
0.44015660357691044
Side Men Running Season 2003 Metric FG3Percent
0.34710929076498276
Side Men Running Season 2003 Metric FTPercent
0.6899986703791553
Side Men Running Season 2003 Metric ORPercent
0.33359629479663866
Side Men Running Season 2003 Metric AstPercent
0.5507801041239485
Side Men Running Season 2003 Metric PointsPerPoss
1.021149226717963
Side Men Running Season 2003 Metric TOPerPoss
0.21743982725874816
Side Men Running Season 2003 Metric StlPerPoss
0.10290550958514241
Side Men Running Season 2003 Metric BlkPerPoss
0.047626236598351177
Side Men Running Season 2003 Metric PFPerPoss
0.051457246923254485
Side Men Running Season 2004 Metric Score
68.98063880988843
Side Men Running Season 2004 Metric Possessions
67.90723692846204
Side Men Running Season 2004 Metric FGPercent
0.43751278246689945
Side Men Running

,TeamID,ScoreCreationRating,ScorePreventionRating,PossessionsCreationRating,PossessionsPreventionRating,FGPercentCreationRating,FGPercentPreventionRating,FG3PercentCreationRating,FG3PercentPreventionRating,FTPercentCreationRating,...,PointsPerPossPreventionRating,TOPerPossCreationRating,TOPerPossPreventionRating,StlPerPossCreationRating,StlPerPossPreventionRating,BlkPerPossCreationRating,BlkPerPossPreventionRating,PFPerPossCreationRating,PFPerPossPreventionRating,Season
0,3103,61.366759,59.473909,67.802821,68.132275,0.399896,0.385480,0.307159,0.324466,0.704287,...,0.874124,0.265313,0.266066,0.114503,0.128972,0.034588,0.056968,0.057374,0.064351,2010
1,3104,67.485805,64.245816,74.928633,74.738010,0.414024,0.389882,0.317300,0.318984,0.631054,...,0.862923,0.248920,0.273242,0.110946,0.112656,0.040563,0.042879,0.055389,0.056478,2010
2,3110,60.325928,57.138246,64.096585,64.236769,0.394734,0.386562,0.313728,0.299410,0.742956,...,0.885765,0.237231,0.250470,0.117907,0.092081,0.057924,0.047597,0.059059,0.045984,2010
3,3111,65.728999,65.163645,73.822272,73.175811,0.429816,0.372665,0.322902,0.325228,0.632450,...,0.891819,0.293600,0.243480,0.128470,0.133006,0.084109,0.050590,0.063936,0.066895,2010
4,3119,51.341335,62.890809,65.310200,65.788437,0.348011,0.398362,0.271117,0.361635,0.681215,...,0.956582,0.265398,0.241836,0.093357,0.124673,0.046659,0.059212,0.046772,0.062033,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5597,3110,53.632489,74.918782,72.109333,72.451984,0.350121,0.428774,0.262366,0.294401,0.652453,...,1.034106,0.250050,0.166896,0.081574,0.137461,0.039728,0.045356,0.040987,0.068468,2025
5598,3115,49.260516,72.411109,73.240779,73.819151,0.333142,0.440405,0.231452,0.332202,0.669737,...,0.980434,0.268321,0.238335,0.111717,0.127915,0.026279,0.055245,0.055958,0.063736,2025
5599,3421,52.331835,76.273440,71.460030,71.438731,0.342575,0.481336,0.247458,0.303916,0.747630,...,1.061108,0.265560,0.208641,0.084269,0.151349,0.027070,0.053701,0.042057,0.075550,2025
5600,3310,54.882207,82.445586,80.019909,79.255330,0.334473,0.495159,0.255307,0.373661,0.633226,...,1.040143,0.269143,0.273840,0.135039,0.146312,0.040678,0.063919,0.067003,0.073215,2025


In [22]:
all_team_women_stat_ratings_df

,TeamID,PossessionsCreationRating,PossessionsPreventionRating,FGPercentCreationRating,FGPercentPreventionRating,FG3PercentCreationRating,FG3PercentPreventionRating,FTPercentCreationRating,FTPercentPreventionRating,ORPercentCreationRating,...,PointsPerPossPreventionRating,TOPerPossCreationRating,TOPerPossPreventionRating,StlPerPossCreationRating,StlPerPossPreventionRating,BlkPerPossCreationRating,BlkPerPossPreventionRating,PFPerPossCreationRating,PFPerPossPreventionRating,Season
0,3103,67.802821,68.132275,0.399896,0.385480,0.307159,0.324466,0.704287,0.689571,0.388352,...,0.874124,0.265313,0.266066,0.114503,0.128972,0.034588,0.056968,0.057374,0.064351,2010
1,3104,74.928633,74.738010,0.414024,0.389882,0.317300,0.318984,0.631054,0.638325,0.359251,...,0.862923,0.248920,0.273242,0.110946,0.112656,0.040563,0.042879,0.055389,0.056478,2010
2,3110,64.096585,64.236769,0.394734,0.386562,0.313728,0.299410,0.742956,0.686044,0.364220,...,0.885765,0.237231,0.250470,0.117907,0.092081,0.057924,0.047597,0.059059,0.045984,2010
3,3111,73.822272,73.175811,0.429816,0.372665,0.322902,0.325228,0.632450,0.702653,0.391751,...,0.891819,0.293600,0.243480,0.128470,0.133006,0.084109,0.050590,0.063936,0.066895,2010
4,3119,65.310200,65.788437,0.348011,0.398362,0.271117,0.361635,0.681215,0.686988,0.313211,...,0.956582,0.265398,0.241836,0.093357,0.124673,0.046659,0.059212,0.046772,0.062033,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5597,3110,72.109333,72.451984,0.350121,0.428774,0.262366,0.294401,0.652453,0.695363,0.227386,...,1.034106,0.250050,0.166896,0.081574,0.137461,0.039728,0.045356,0.040987,0.068468,2025
5598,3115,73.240779,73.819151,0.333142,0.440405,0.231452,0.332202,0.669737,0.686645,0.231459,...,0.980434,0.268321,0.238335,0.111717,0.127915,0.026279,0.055245,0.055958,0.063736,2025
5599,3421,71.460030,71.438731,0.342575,0.481336,0.247458,0.303916,0.747630,0.719518,0.267893,...,1.061108,0.265560,0.208641,0.084269,0.151349,0.027070,0.053701,0.042057,0.075550,2025
5600,3310,80.019909,79.255330,0.334473,0.495159,0.255307,0.373661,0.633226,0.752468,0.232478,...,1.040143,0.269143,0.273840,0.135039,0.146312,0.040678,0.063919,0.067003,0.073215,2025


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14
